In [84]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import log_loss

In [85]:
train_df = pd.read_csv("input/train.csv")
test_df = pd.read_csv("input/test.csv")
sample_df = pd.read_csv("input/sample_submission.csv")

In [86]:
train_df

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114316,228708,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AL,NaN,NaN,NaN,0,NaN,NaN
114317,228710,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,E,NaN,NaN,NaN,1,NaN,NaN
114318,228711,1,NaN,NaN,C,NaN,10.069277,NaN,NaN,0.323324,...,NaN,NaN,0.156764,Q,NaN,NaN,2.417606,2,NaN,NaN
114319,228712,1,NaN,NaN,C,NaN,10.106144,NaN,NaN,0.309226,...,NaN,NaN,0.490658,BW,NaN,NaN,3.526650,0,NaN,NaN


In [87]:
test_df

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,0,1.375465e+00,11.361141,C,4.200778,6.577000,2.081784,1.784386,0.011094,9.523810,...,7.619048,1.815241,1.112270e-07,AF,1.292368,3.903345,1.485925,0,2.333334,1.428572e+00
1,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,I,NaN,NaN,NaN,0,NaN,NaN
2,2,-4.903407e-07,8.201529,C,4.544371,6.550100,1.558442,2.467532,0.007164,7.142858,...,5.714286,1.970928,1.412265e-02,AV,1.128724,5.844156,1.475892,0,1.263157,-6.380022e-07
3,7,2.661870e+00,3.041241,C,1.657216,9.773080,2.078337,1.430855,1.252157,7.959596,...,4.404040,8.163614,1.100329e+00,B,1.988688,1.558753,2.448814,0,5.385474,1.493777e+00
4,10,1.252822e+00,11.283352,C,4.638388,8.520510,2.302484,3.510159,0.074263,7.612904,...,6.580644,1.325654,2.584588e-01,A,1.863796,2.666478,2.374275,0,0.681672,2.264151e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114388,228700,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AG,NaN,NaN,NaN,0,NaN,NaN
114389,228703,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,H,NaN,NaN,NaN,0,NaN,NaN
114390,228704,1.523740e+00,6.571597,C,3.827086,8.824122,2.635660,2.218992,6.534959,8.852942,...,6.852941,2.093560,5.111015e+00,V,1.826453,2.661700,1.601567,0,1.816594,1.634616e+00
114391,228706,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AZ,NaN,NaN,NaN,0,NaN,NaN


In [88]:
sample_df

,ID,PredictedProb
0,0,0.5
1,1,0.5
2,2,0.5
3,7,0.5
4,10,0.5
...,...,...
114388,228700,0.5
114389,228703,0.5
114390,228704,0.5
114391,228706,0.5


In [89]:
train_test_df = pd.concat([train_df, test_df], axis=0)
train_test_df

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1.0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,3.575369e-02,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1.0,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,5.988956e-01,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1.0,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,1.345191e-02,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1.0,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,2.267384e-03,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1.0,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114388,228700,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AG,NaN,NaN,NaN,0,NaN,NaN
114389,228703,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,H,NaN,NaN,NaN,0,NaN,NaN
114390,228704,NaN,1.523740,6.571597,C,3.827086,8.824122,2.635660,2.218992,6.534959,...,6.852941,2.093560,5.111015e+00,V,1.826453,2.661700,1.601567,0,1.816594,1.634616
114391,228706,NaN,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,AZ,NaN,NaN,NaN,0,NaN,NaN


In [90]:
train_test_df.dtypes

ID          int64
target    float64
v1        float64
v2        float64
v3         object
           ...   
v127      float64
v128      float64
v129        int64
v130      float64
v131      float64
Length: 133, dtype: object

In [91]:
# カテゴリカル変数の列を特定
cat_cols = train_test_df.select_dtypes(include=['object']).columns.tolist()

for col in cat_cols:
    unique_values = train_test_df[col].dropna().unique()

    le = LabelEncoder()
    le.fit(unique_values)

    # エンコーディングされたデータで元の列を置き換え（NaNはそのまま保持）
    train_test_df.loc[train_test_df[col].notna(), col] = le.transform(train_test_df[col].dropna())

    # エンコーディング後のカテゴリ値を int 型に変換
    train_test_df[col] = train_test_df[col].astype('Int64')  # 'Int64'はNaNを含む整数データ型

In [92]:
train_test_df

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1.0,1.335739,8.727474,2,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,3.575369e-02,21,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1.0,NaN,NaN,2,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,5.988956e-01,6,NaN,NaN,1.957825,0,NaN,NaN
2,5,1.0,0.943877,5.310079,2,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,1.345191e-02,5,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1.0,0.797415,8.304757,2,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,2.267384e-03,64,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1.0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,89,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114388,228700,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7,NaN,NaN,NaN,0,NaN,NaN
114389,228703,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,71,NaN,NaN,NaN,0,NaN,NaN
114390,228704,NaN,1.523740,6.571597,2,3.827086,8.824122,2.635660,2.218992,6.534959,...,6.852941,2.093560,5.111015e+00,85,1.826453,2.661700,1.601567,0,1.816594,1.634616
114391,228706,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26,NaN,NaN,NaN,0,NaN,NaN


In [93]:
train_test_df.dtypes

ID          int64
target    float64
v1        float64
v2        float64
v3          Int64
           ...   
v127      float64
v128      float64
v129        int64
v130      float64
v131      float64
Length: 133, dtype: object

In [94]:
train_test_df_2 = train_test_df.drop('ID', axis=1)
train_df_2 = train_test_df[np.isfinite(train_test_df["target"])]
test_df_2 = train_test_df[np.isnan(train_test_df["target"])].drop("target", axis=1)

In [95]:
train_df_2

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1.0,1.335739,8.727474,2,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,21,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1.0,NaN,NaN,2,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,6,NaN,NaN,1.957825,0,NaN,NaN
2,5,1.0,0.943877,5.310079,2,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,5,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1.0,0.797415,8.304757,2,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,64,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1.0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,89,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114316,228708,1.0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,12,NaN,NaN,NaN,0,NaN,NaN
114317,228710,1.0,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,68,NaN,NaN,NaN,1,NaN,NaN
114318,228711,1.0,NaN,NaN,2,NaN,10.069277,NaN,NaN,0.323324,...,NaN,NaN,0.156764,80,NaN,NaN,2.417606,2,NaN,NaN
114319,228712,1.0,NaN,NaN,2,NaN,10.106144,NaN,NaN,0.309226,...,NaN,NaN,0.490658,50,NaN,NaN,3.526650,0,NaN,NaN


In [96]:
test_df_2

,ID,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,0,1.375465e+00,11.361141,2,4.200778,6.577000,2.081784,1.784386,0.011094,9.523810,...,7.619048,1.815241,1.112270e-07,6,1.292368,3.903345,1.485925,0,2.333334,1.428572e+00
1,1,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,72,NaN,NaN,NaN,0,NaN,NaN
2,2,-4.903407e-07,8.201529,2,4.544371,6.550100,1.558442,2.467532,0.007164,7.142858,...,5.714286,1.970928,1.412265e-02,22,1.128724,5.844156,1.475892,0,1.263157,-6.380022e-07
3,7,2.661870e+00,3.041241,2,1.657216,9.773080,2.078337,1.430855,1.252157,7.959596,...,4.404040,8.163614,1.100329e+00,27,1.988688,1.558753,2.448814,0,5.385474,1.493777e+00
4,10,1.252822e+00,11.283352,2,4.638388,8.520510,2.302484,3.510159,0.074263,7.612904,...,6.580644,1.325654,2.584588e-01,0,1.863796,2.666478,2.374275,0,0.681672,2.264151e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114388,228700,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,7,NaN,NaN,NaN,0,NaN,NaN
114389,228703,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,71,NaN,NaN,NaN,0,NaN,NaN
114390,228704,1.523740e+00,6.571597,2,3.827086,8.824122,2.635660,2.218992,6.534959,8.852942,...,6.852941,2.093560,5.111015e+00,85,1.826453,2.661700,1.601567,0,1.816594,1.634616e+00
114391,228706,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,26,NaN,NaN,NaN,0,NaN,NaN


In [97]:
train_X, train_y = train_df_2.drop("target", axis=1), train_df_2["target"]

In [98]:
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.2, shuffle=True)

In [99]:
dtrain = lgb.Dataset(train_X, train_y)
dvalid = lgb.Dataset(valid_X, valid_y)

In [100]:
lgb_params = {
    'boosting':'gbdt',
    'objective' : 'binary',
    'learning_rate' : 0.1,
    'num_leaves' : 64,
    'max_depth' : 10,
    'min_data_in_leaf' : 20,
    'bagging_fraction' : 0.8,
    'bagging_freq' : 1,
    'feature_fraction' : 0.9,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    'verbose' : -1,
    'force_row_wise': True
}

In [101]:
# LightGBM modelの学習
# 早期終了のためのコールバックを定義
early_stopping = lgb.early_stopping(stopping_rounds=10, first_metric_only=True, verbose=True)
verbose_eval = lgb.log_evaluation(1)
light_gbm = lgb.train(
    params=lgb_params,
    train_set=dtrain,
    valid_sets=[dvalid],
    num_boost_round=10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
    callbacks=[early_stopping, verbose_eval] 
)

[1]	valid_0's binary_logloss: 0.537593
Training until validation scores don't improve for 10 rounds
[2]	valid_0's binary_logloss: 0.528246
[3]	valid_0's binary_logloss: 0.521743
[4]	valid_0's binary_logloss: 0.515246
[5]	valid_0's binary_logloss: 0.508485
[6]	valid_0's binary_logloss: 0.503031
[7]	valid_0's binary_logloss: 0.498518
[8]	valid_0's binary_logloss: 0.494662
[9]	valid_0's binary_logloss: 0.49122
[10]	valid_0's binary_logloss: 0.488929
[11]	valid_0's binary_logloss: 0.486487
[12]	valid_0's binary_logloss: 0.484397
[13]	valid_0's binary_logloss: 0.482476
[14]	valid_0's binary_logloss: 0.480827
[15]	valid_0's binary_logloss: 0.479474
[16]	valid_0's binary_logloss: 0.478411
[17]	valid_0's binary_logloss: 0.477502
[18]	valid_0's binary_logloss: 0.476662
[19]	valid_0's binary_logloss: 0.475971
[20]	valid_0's binary_logloss: 0.475374
[21]	valid_0's binary_logloss: 0.474818
[22]	valid_0's binary_logloss: 0.474269
[23]	valid_0's binary_logloss: 0.473883
[24]	valid_0's binary_logloss

In [102]:
pred_test = light_gbm.predict(test_df_2, num_iteration=light_gbm.best_iteration)

In [103]:
sub_df = pd.DataFrame({
    "ID": test_df["ID"],
    "PredictedProb": pred_test
})

In [104]:
sub_df.to_csv("submission.csv", index=False)